In [1]:
"""
01_basic_dataclasses.py - Basic usage of dataclasses

This file demonstrates the proper use of dataclasses for simple data structures.
"""

from dataclasses import dataclass, field
from typing import List, Optional


# GOOD EXAMPLE: Simple dataclass with type hints
@dataclass
class Person:
    name: str
    age: int
    email: Optional[str] = None
    # Using field() with default_factory for mutable default values
    tags: List[str] = field(default_factory=list)

    def is_adult(self) -> bool:
        """Example method that uses the dataclass attributes."""
        return self.age >= 18


# Usage example
def demo_good_usage():
    # Creating instances
    person1 = Person(name="Alice", age=30, email="alice@example.com")
    person2 = Person(name="Bob", age=25)
    person3 = Person(name="Charlie", age=17, tags=["student", "part-time"])

    # Adding to a mutable field
    person1.tags.append("developer")

    # Using the built-in string representation
    print(f"Person 1: {person1}")
    print(f"Person 2: {person2}")
    print(f"Person 3: {person3}")

    # Using the instance method
    print(f"Is {person1.name} an adult? {person1.is_adult()}")
    print(f"Is {person3.name} an adult? {person3.is_adult()}")


# BAD EXAMPLE: Class without dataclass
class PersonBad:
    def __init__(self, name, age, email=None, tags=None):
        self.name = name
        self.age = age
        self.email = email
        # Common mistake: mutable default
        self.tags = tags if tags is not None else []

    # Have to manually define string representation
    def __repr__(self):
        return f"PersonBad(name={self.name}, age={self.age}, email={self.email}, tags={self.tags})"

    # Have to manually define equality
    def __eq__(self, other):
        if not isinstance(other, PersonBad):
            return False
        return (self.name == other.name and
                self.age == other.age and
                self.email == other.email and
                self.tags == other.tags)


def demo_bad_usage():
    # More verbose and error-prone without dataclasses
    person1 = PersonBad("Alice", 30, "alice@example.com")
    person2 = PersonBad("Bob", 25)

    print(f"PersonBad 1: {person1}")
    print(f"PersonBad 2: {person2}")


if __name__ == "__main__":
    print("=== GOOD DATACLASS EXAMPLES ===")
    demo_good_usage()

    print("\n=== BAD REGULAR CLASS EXAMPLES ===")
    demo_bad_usage()

=== GOOD DATACLASS EXAMPLES ===
Person 1: Person(name='Alice', age=30, email='alice@example.com', tags=['developer'])
Person 2: Person(name='Bob', age=25, email=None, tags=[])
Person 3: Person(name='Charlie', age=17, email=None, tags=['student', 'part-time'])
Is Alice an adult? True
Is Charlie an adult? False

=== BAD REGULAR CLASS EXAMPLES ===
PersonBad 1: PersonBad(name=Alice, age=30, email=alice@example.com, tags=[])
PersonBad 2: PersonBad(name=Bob, age=25, email=None, tags=[])


In [2]:
"""
02_nested_dataclasses.py - Working with nested dataclasses

This file demonstrates how to properly structure and work with nested dataclasses.
"""

from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional
import json


# GOOD EXAMPLE: Well-structured nested dataclasses
@dataclass
class Address:
    street: str
    city: str
    state: str
    zip_code: str
    country: str = "USA"


@dataclass
class Contact:
    email: str
    phone: Optional[str] = None


@dataclass
class Employee:
    id: int
    name: str
    department: str
    # Nested dataclass as a field
    address: Address
    # Another nested dataclass
    contact: Contact
    # List of another dataclass type
    skills: List[str] = field(default_factory=list)

    def to_json(self) -> str:
        """Convert the employee data to JSON string."""
        # asdict recursively converts dataclasses to dictionaries
        return json.dumps(asdict(self), indent=2)

    def add_skill(self, skill: str) -> None:
        """Add a skill to the employee's skill list."""
        if skill not in self.skills:
            self.skills.append(skill)


# Usage example - Good pattern
def demo_good_nested():
    # Create nested dataclass instances
    address = Address(
        street="123 Tech Lane",
        city="San Francisco",
        state="CA",
        zip_code="94107"
    )

    contact = Contact(
        email="john.doe@example.com",
        phone="555-123-4567"
    )

    # Create the parent dataclass with nested instances
    employee = Employee(
        id=1001,
        name="John Doe",
        department="Engineering",
        address=address,
        contact=contact,
        skills=["Python", "Data Science"]
    )

    # Access nested attributes with proper dot notation
    print(f"Employee: {employee.name}")
    print(f"City: {employee.address.city}")
    print(f"Email: {employee.contact.email}")

    # Add a skill
    employee.add_skill("Machine Learning")
    print(f"Skills: {employee.skills}")

    # Convert to JSON
    print("\nEmployee JSON:")
    print(employee.to_json())


# BAD EXAMPLE: Poorly structured data without proper nesting
@dataclass
class EmployeeBad:
    id: int
    name: str
    department: str
    # Flat structure instead of proper nesting
    street: str
    city: str
    state: str
    zip_code: str
    email: str
    # Fields with default values must come after required fields
    phone: Optional[str] = None
    country: str = "USA"
    skills: List[str] = field(default_factory=list)


# Even worse example: using dictionaries instead of proper dataclasses
class EmployeeWorse:
    def __init__(self, id, name, department, address_dict, contact_dict, skills=None):
        self.id = id
        self.name = name
        self.department = department
        # Using dictionaries instead of proper dataclasses
        self.address = address_dict  # {"street": "...", "city": "...", ...}
        self.contact = contact_dict  # {"email": "...", "phone": "..."}
        self.skills = skills or []


def demo_bad_nested():
    # Flat structure makes it harder to organize and maintain
    employee_bad = EmployeeBad(
        id=1001,
        name="John Doe",
        department="Engineering",
        street="123 Tech Lane",
        city="San Francisco",
        state="CA",
        zip_code="94107",
        email="john.doe@example.com",
        phone="555-123-4567",
        skills=["Python", "Data Science"]
    )

    print("\n=== BAD FLAT STRUCTURE ===")
    print(f"Employee: {employee_bad}")

    # Using dictionaries is even worse
    employee_worse = EmployeeWorse(
        id=1001,
        name="John Doe",
        department="Engineering",
        address_dict={
            "street": "123 Tech Lane",
            "city": "San Francisco",
            "state": "CA",
            "zip_code": "94107"
        },
        contact_dict={
            "email": "john.doe@example.com",
            "phone": "555-123-4567"
        },
        skills=["Python", "Data Science"]
    )

    print("\n=== WORSE DICTIONARY APPROACH ===")
    # No nice string representation
    print(f"Employee: {employee_worse.__dict__}")
    # Error-prone access to nested data
    print(f"City: {employee_worse.address['city']}")


if __name__ == "__main__":
    print("=== GOOD NESTED DATACLASS EXAMPLE ===")
    demo_good_nested()

    print("\n=== BAD NESTED DATACLASS EXAMPLES ===")
    demo_bad_nested()

=== GOOD NESTED DATACLASS EXAMPLE ===
Employee: John Doe
City: San Francisco
Email: john.doe@example.com
Skills: ['Python', 'Data Science', 'Machine Learning']

Employee JSON:
{
  "id": 1001,
  "name": "John Doe",
  "department": "Engineering",
  "address": {
    "street": "123 Tech Lane",
    "city": "San Francisco",
    "state": "CA",
    "zip_code": "94107",
    "country": "USA"
  },
  "contact": {
    "email": "john.doe@example.com",
    "phone": "555-123-4567"
  },
  "skills": [
    "Python",
    "Data Science",
    "Machine Learning"
  ]
}

=== BAD NESTED DATACLASS EXAMPLES ===

=== BAD FLAT STRUCTURE ===
Employee: EmployeeBad(id=1001, name='John Doe', department='Engineering', street='123 Tech Lane', city='San Francisco', state='CA', zip_code='94107', email='john.doe@example.com', phone='555-123-4567', country='USA', skills=['Python', 'Data Science'])

=== WORSE DICTIONARY APPROACH ===
Employee: {'id': 1001, 'name': 'John Doe', 'department': 'Engineering', 'address': {'street': 